## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,33.46,100,100,19.57,sleet
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,17,8.01,few clouds
2,2,Chapeco,BR,-27.0964,-52.6183,78.55,47,20,11.50,few clouds
3,3,Jamestown,US,42.0970,-79.2353,84.06,61,75,10.36,broken clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,65.07,64,100,5.70,overcast clouds


In [55]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [56]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

City_ID                553
City                   553
Country                549
Lat                    553
Lng                    553
Max Temp               553
Humidity               553
Cloudiness             553
Wind Speed             553
Current Description    553
dtype: int64

In [57]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                553
City                   553
Country                549
Lat                    553
Lng                    553
Max Temp               553
Humidity               553
Cloudiness             553
Wind Speed             553
Current Description    553
dtype: int64

In [67]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,17,8.01,few clouds
2,2,Chapeco,BR,-27.0964,-52.6183,78.55,47,20,11.50,few clouds
3,3,Jamestown,US,42.0970,-79.2353,84.06,61,75,10.36,broken clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,65.07,64,100,5.70,overcast clouds
5,5,Hofn,IS,64.2539,-15.2082,53.62,87,75,8.05,broken clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Hargeysa,SO,9.5600,44.0650,72.55,51,2,26.11,clear sky
666,666,Daru,PG,-9.0763,143.2092,75.72,92,30,10.18,scattered clouds
667,667,Yeppoon,AU,-23.1333,150.7333,56.98,82,27,10.04,scattered clouds
668,668,Panambi,BR,-28.2925,-53.5017,72.52,75,100,8.70,overcast clouds


In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,80.56,few clouds,-0.7393,-90.3518,
2,Chapeco,BR,78.55,few clouds,-27.0964,-52.6183,
3,Jamestown,US,84.06,broken clouds,42.0970,-79.2353,
4,Port Alfred,ZA,65.07,overcast clouds,-33.5906,26.8910,
5,Hofn,IS,53.62,broken clouds,64.2539,-15.2082,
6,Arraial Do Cabo,BR,80.56,clear sky,-22.9661,-42.0278,
8,Talnakh,RU,67.21,overcast clouds,69.4865,88.3972,
9,Thompson,CA,55.56,overcast clouds,55.7435,-97.8558,
10,Moose Factory,CA,73.40,overcast clouds,51.2585,-80.6094,
13,Rzhev,RU,62.06,clear sky,56.2624,34.3282,


In [79]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
             

Hotel not found... skipping.40
Hotel not found... skipping.53
Hotel not found... skipping.75
Hotel not found... skipping.109
Hotel not found... skipping.126
Hotel not found... skipping.135
Hotel not found... skipping.138
Hotel not found... skipping.145
Hotel not found... skipping.163
Hotel not found... skipping.167
Hotel not found... skipping.180
Hotel not found... skipping.225
Hotel not found... skipping.232
Hotel not found... skipping.234
Hotel not found... skipping.255
Hotel not found... skipping.256
Hotel not found... skipping.259
Hotel not found... skipping.264
Hotel not found... skipping.271
Hotel not found... skipping.284
Hotel not found... skipping.291
Hotel not found... skipping.326
Hotel not found... skipping.341
Hotel not found... skipping.342
Hotel not found... skipping.351
Hotel not found... skipping.414
Hotel not found... skipping.417
Hotel not found... skipping.426
Hotel not found... skipping.431
Hotel not found... skipping.443
Hotel not found... skipping.445
Hotel not f

In [80]:
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,80.56,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Chapeco,BR,78.55,few clouds,-27.0964,-52.6183,Golden Hotel
3,Jamestown,US,84.06,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
4,Port Alfred,ZA,65.07,overcast clouds,-33.5906,26.8910,The Halyards Hotel
5,Hofn,IS,53.62,broken clouds,64.2539,-15.2082,Hótel Höfn
...,...,...,...,...,...,...,...
664,Hargeysa,SO,72.55,clear sky,9.5600,44.0650,City Center Hotel
666,Daru,PG,75.72,scattered clouds,-9.0763,143.2092,Daru Lodge
667,Yeppoon,AU,56.98,scattered clouds,-23.1333,150.7333,Villa Mar Colina
668,Panambi,BR,72.52,overcast clouds,-28.2925,-53.5017,Hotel Lages Panambi


In [84]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [82]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [83]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))